In [1]:
import pandas as pd
from import_data import import_training_data
import numpy as np
from sklearn.model_selection import train_test_split
import time
import xgboost
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold


In [3]:
# importing and splitting formatted data

df = import_training_data()
label = df.pop('finalLapTime')

X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.4, random_state=42)

X_train.head()


lastLapTime  currentLapTime  currentLapNum  lapDistance  carPosition  \
index                                                                          
1723       0.000000       18.279922            1.0   944.324536          8.0   
79738     95.424316       59.883719           11.0  3745.617969         15.0   
133875    93.812622       56.579514           18.0  3496.356250          7.0   
168520    94.938599       74.946606           22.0  4779.342188         18.0   
7517       0.000000       90.053277            1.0  5381.325977          8.0   

        worldPositionX  worldPositionY  worldPositionZ  worldVelocityX  \
index                                                                    
1723        144.236038       -1.113423       -9.127578       18.883812   
79738       375.873450       -0.694183     -132.638988        6.145464   
133875      319.306647        0.923111     -369.686780        4.880086   
168520       35.709633       -6.960154      762.818506      -36.604169   
7517       -344.313553       -8.013516      768.434247      -46.673740   

        worldVelocityY  ...  summed_tyresSurfaceTemperatureRL  \
index                   ...                                     
1723          0.077933  ...                            162651   
79738         0.002611  ...                            421955   
133875       -0.349835  ...                            400745   
168520       -0.830729  ...                            536561   
7517          0.552251  ...                            715187   

        summed_tyresSurfaceTemperatureRR  summed_tyresSurfaceTemperatureFL  \
index                                                                        
1723                              161395                            147963   
79738                             412665                            438489   
133875                            392464                            414110   
168520                            524296                            555013   
7517                              701754                            721881   

        summed_tyresSurfaceTemperatureFR  summed_engineTemperature  \
index                                                                
1723                              145350                    164318   
79738                             420357                    501940   
133875                            397092                    485010   
168520                            528509                    634903   
7517                              688731                    773137   

        summed_tyresWearRL  summed_tyresWearRR  summed_tyresWearFL  \
index                                                                
1723                     0                   0                   0   
79738                31299               26656               39638   
133875               68947               59573               80884   
168520              156745              134298              178491   
7517                  7470                6702                7471   

        summed_tyresWearFR  summed_carPosition  
index                                           
1723                     0               16347  
79738                30556               68667  
133875               60942               30471  
168520              134930              104202  
7517                  5391               66086  

[5 rows x 78 columns]

In [4]:
#for tuning parameters
parameters_for_testing = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0,0.03,0.1,0.3],
   'min_child_weight':[1.5,6,10],
   'learning_rate':[0.1,0.07],
   'max_depth':[3,5],
   'n_estimators':[500],
   'subsample':[0.6,0.95]
}


Now we want to run a grid search on the parameters defined above to find the optimal parameters for our XgBoost model

In [5]:


xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
    min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,tree_method='gpu_hist', nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')
gsearch1.fit(X_train,y_train)




Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   49.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   52.0s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  3.4min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  6.7min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  8.9min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed: 10.5min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed: 12.7min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed: 14.7min
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed: 17.1min
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed: 20.5min
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed: 22.8min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed: 25.2min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed: 29.2min
[Parallel(

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.8,
                                    gamma=0, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=0.1, max_delta_step=None,
                                    max_depth=5, min_child_weight=1,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_jobs=None, nthread=6,...
                                    scale_pos_weight=1, seed=27, subsample=0.8,
                                    tree_method='gpu_hist',
                                    validate_parameters=None, verbosity=None),
             iid=False, n_jobs=6,
             param_grid={'colsample_bytree': [0.4, 0

In [6]:
gsearch1.best_params_



{'colsample_bytree': 0.8,
 'gamma': 0.03,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 1.5,
 'n_estimators': 500,
 'subsample': 0.95}

# Best Model Parameters

{'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 1.5,
 'n_estimators': 500,
 'subsample': 0.6}

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.2, random_state=42)


tuned_model = xgboost.XGBRegressor(colsample_bytree = 0.8, gamma=0.03, learning_rate = 0.1, max_depth = 5,
                                   min_child_weight = 1.5, n_estimators = 10000, subsample=0.95, 
                                   tree_method = 'gpu_hist')

tuned_model.fit(X_train, y_train)
pred = tuned_model.predict(X_test)
err = mae(y_test, pred)
display(err)

0.015731636340799496

In [8]:
import pickle
file_name = 'xgboost_model.pkl'
pickle.dump(tuned_model, open(file_name, 'wb'))

In [9]:
from time import time
s = time()
pred = tuned_model.predict(X_test)
print(time()-s)
err = mae(y_test, pred)
display(err)

0.7671542167663574


0.015731636340799496